This script will be for linking the siteIDs to the catchment IDs - which are either NHDPlusIDs,
gridcodes, or new catIDs made from the gridcodes for BB. This will give us a way to link the
thermal sensitivity responses by year to the covariates. This script also includes code to
check the watersheds. The watersheds are individual feature classes to avoid overlapping
polygons so the best way to check they are correct is to 1) check that every siteID/catID
has a watershed, and 2) check that the watershed area approximately matches the watershed
area from the flow accumulation grid. They won't be exact for sites that are not close to
the catchment outlet and could be quite different for very small watersheds in some cases.

# Watershed Summaries

1. read in all watersheds feature classes
2. create a table with the NHDPlusID/catID of the watershed name, region, and watershed area
3. merge original point feature classes (inside and outside bb)
4. do a spatial join with merged catchments by region to get catID and region on the points dataset
5. clip to the region and save in each regional gdb
6. merge all point feature classes with siteIDs and catIDs into one table
7. join the watershed area from the watershed fcs to the points table using the catID
NOT DONE and not sure we need it: 8. join the watershed area from the fac grid to the points table using the catID
9. compare the results to ensure that watersheds have been created correctly

In [ ]:
# steps 1 and 2

import arcpy
import os
import time

import numpy
import pandas as pd

regions = ["Kodiak", "Copper_River", "Prince_William_Sound", "Cook_Inlet", "Bristol_Bay"]

wtdList = []

for region in regions:
    local_gdb = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb\\Watersheds"
    arcpy.env.workspace = local_gdb
    wtds = arcpy.ListFeatureClasses()
    print(region + ": " + str(len(wtds)) + " watersheds")

    for wtd in wtds:
        wtdName = wtd[4:20]
        print("Starting wtd: " + wtdName)
        wtdPath = os.path.join(arcpy.env.workspace, wtd)
        # field_names = [f.name for f in arcpy.ListFields(wtdPath)]
        # print(field_names)
        # if "Area_km2" in field_names:
        #     print("Area already calculated")
        # else:
        #     arcpy.AddField_management(wtdPath, "Area_km2", "DOUBLE")
        #     expression1 = "{0}".format("!SHAPE.area@SQUAREKILOMETERS!")
        #     arcpy.CalculateField_management(wtdPath, "Area_km2", expression1, "PYTHON", )
        wtdArea = [row[0] for row in arcpy.da.SearchCursor(wtdPath, ['Area_km2'])]
        # print("wtdName: " + str(wtdArea))
        wtdList.append({'Region': region, 'cat_ID': wtdName, 'Area_km2': wtdArea})

wtdDf = pd.DataFrame(wtdList)
print(wtdDf)


In [1]:
#merge watersheds into one feature class for each region

import arcpy
import os
import pandas as pd

arcpy.env.overwriteOutput = True

regions = ["Kodiak", "Copper_River", "Prince_William_Sound", "Cook_Inlet", "Bristol_Bay"]
regions = ["Kodiak", "Prince_William_Sound"]

for region in regions:
    local_gdb = "W:\\GIS\\AKSSF\\" + region + "\\" + region + ".gdb"
    arcpy.env.workspace = local_gdb + "\\Watersheds"
    wtds = arcpy.ListFeatureClasses()
    print(region + ": " + str(len(wtds)) + " watersheds")

    #add cat_ID to each watershed before merging
    for wtd in wtds:
        wtdName = wtd[4:20]
        wtdPath = os.path.join(arcpy.env.workspace, wtd)
        arcpy.AddField_management(wtdPath, "cat_ID", "DOUBLE")
        # expression1 = "{0}".format("!SHAPE.area@SQUAREKILOMETERS!")
        arcpy.CalculateField_management(wtdPath, "cat_ID", wtdName, "PYTHON")

    arcpy.env.workspace = local_gdb
    wtdMerge = [local_gdb + "\\Watersheds\\" + s for s in wtds]
    wtd_output = "wtds_merge"
    arcpy.Merge_management(wtdMerge, wtd_output)


Kodiak: 28 watersheds
Prince_William_Sound: 19 watersheds


In [ ]:
print(len(wtdDf))
print(len(wtdList))


# step 3 - done

import arcpy

gdb = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb"
arcpy.env.workspace = gdb
bb_pts = gdb + "\\bb_md_verified_DM"
other_pts = gdb + "\\sites_outside_bb_verified_DM"

output = "akssf_pts_verified"
arcpy.Merge_management([bb_pts, other_pts], output)

In [ ]:
# steps 4-6

import arcpy
import os

arcpy.env.overwriteOutput = True
regions_dict = {"Kodiak": '!gridcode!', "Copper_River": '!NHDPlusID!', "Prince_William_Sound": '!gridcode!',
                "Cook_Inlet": '!NHDPlusID!', "Bristol_Bay": '!catID!'}
# regions_dict = {"Bristol_Bay": '!catID!'}

points = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\akssf_pts_verified"
# sites_lst = []
cats_lst = []

for key, value in regions_dict.items():
    arcpy.env.workspace = "W:\\GIS\\AKSSF\\" + key + "\\" + key + ".gdb"
    cats = os.path.join(arcpy.env.workspace, "cats_merge")
    # arcpy.Clip_analysis(points, cats, r"memory\sites_clip")
    # arcpy.SpatialJoin_analysis(r"memory\sites_clip", cats, "sites_sj")
    # # #note that catID is in the BB cats_merge, but only a LONG, need a DOUBLE to account for NHDPlusIDs
    # arcpy.AddField_management("sites_sj", "cat_ID", "DOUBLE")
    # arcpy.CalculateField_management("sites_sj", "cat_ID", value)
    # sites_fc = os.path.join(arcpy.env.workspace, "sites_sj")
    # count = arcpy.GetCount_management(sites_fc)
    # print('{} has {} records'.format(sites_fc, count[0]))
    # sites_lst.append(sites_fc)
    cats_lst.append(cats)

#print(sites_lst)
cats_outfile = "W:\\GIS\\AKSSF\\AKSSF_Hydrography.gdb\\all_cats"
arcpy.Merge_management(cats_lst, cats_outfile)

## DM - Picked up here and modified to match local data
Also want to intersect sites with maximum flow accumulation in small (30 meters) buffer.  Buffers need to be generated
iteratively and clipped to their respective catchment to ensure they do not cross pull values from an adjacent catchment.
Need to join stream order, stream slope, and upstream downstream km info from vaa/streams merge datasets.


In [ ]:
import arcpy
import pandas as pd
import os

# Setting outgdb to location created in Covariate notebook but any temp gdb will work
outgdb = r"D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb"
# Set data dir = to folder with regional subfolders and gdbs
data_dir = r"D:\GIS_Temp\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions = arcpy.ListWorkspaces(workspace_type="Folder")

# Create lists to store paths to data copies to be merged
site_lst = []
cats_lst = []
vaas_lst = []
streams_lst = []
wtds_lst = []

# Walk through gdbs and select sites_sj, streams and vaas to merge into single fcs
# Seperate data by
nhdplus_dat = ['Cook_Inlet', 'Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']
# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat
for roi in rois:
    # Loop through regional folders
    for region in regions:
        if roi in str(region):
            if roi in nhdplus_dat:
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID', "cat_ID_con"]
                sj_cur_fields = ['cat_ID_txt', 'cat_ID', "cat_ID_con"]
                wtd_cur_fields = ['cat_ID_txt', 'cat_ID', "cat_ID_con"]
                str_cur_fields = ['cat_ID_txt', 'NHDPlusID', "cat_ID_con"]
                print(
                    f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {sj_cur_fields}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                if roi == 'Bristol_Bay':
                    # Fields for update cursor
                    cat_cur_fields = ['cat_ID_txt', 'catID', "cat_ID_con"]
                    sj_cur_fields = ['cat_ID_txt', 'cat_ID', "cat_ID_con"]
                    wtd_cur_fields = ['cat_ID_txt', 'cat_ID', "cat_ID_con"]
                    str_cur_fields = ['cat_ID_txt', 'catID', "cat_ID_con"]
                else:
                    cat_cur_fields = ['cat_ID_txt', 'gridcode', "cat_ID_con"]
                    wtd_cur_fields = ['cat_ID_txt', 'gridcode', "cat_ID_con"]
                    sj_cur_fields = ['cat_ID_txt', 'gridcode', "cat_ID_con"]
                    str_cur_fields = ['cat_ID_txt', 'LINKNO', "cat_ID_con"]

                print(
                    f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {sj_cur_fields}')
                print('----------')
            print(f'{roi} using data from {region} folder')
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype=['FeatureClass', 'Table'])
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    # Grab sites
                    if filename == 'sites_sj':
                        print(f'{dirpath},{filename}')
                        sitesource = os.path.join(dirpath, filename)
                        sitename = roi + '_' + filename
                        sitepath = os.path.join(outgdb, sitename)
                        # Make local copy projected in AKAlbers
                        if not arcpy.Exists(sitepath):
                            print(f'Copying {sitename}...')
                            print('----------')
                            sitecopy = arcpy.FeatureClassToFeatureClass_conversion(sitesource, outgdb, sitename)
                        else:
                            print(f'{sitepath} already created')
                            sitecopy = sitepath
                            print('----------')
                        site_lst.append(sitecopy)
                        sitefieldnames = []
                        sitelstFields = arcpy.ListFields(sitecopy)
                        for field in sitelstFields:
                            sitefieldnames.append(field.name)
                        if str(sj_cur_fields[0]) in sitefieldnames:
                            print(f'{sj_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print(f'Adding {sj_cur_fields[0]} field to watershed dataset {sitecopy}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(sitecopy, str(sj_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(sitecopy, sj_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0', '')
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
                        if str(sj_cur_fields[2]) in sitefieldnames:
                            print(f'{sj_cur_fields[2]} field already in dataset {sitecopy}')
                            print('----------')
                        else:
                            print(f'Adding {sj_cur_fields[2]} field to watershed dataset {sitecopy}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(sitecopy, str(sj_cur_fields[2]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(sitecopy, sj_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) + '_' + strval.replace('.0', '')
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
                    # Set merged streams dataset
                    elif 'streams_merge' == filename:
                        strsource = os.path.join(dirpath, filename)
                        strname = roi + '_' + filename
                        strpath = os.path.join(outgdb, strname)

                        if not arcpy.Exists(strpath):
                            print(f'Copying {strname}...')
                            print('----------')
                            # Make local copy projected in AKAlbers
                            str_merge = arcpy.FeatureClassToFeatureClass_conversion(strsource, outgdb, strname)
                        else:
                            print(f'Merged streams dataset {strpath} already created...')
                            print('----------')
                            str_merge = strpath
                        streams_lst.append(str_merge)
                        strfieldnames = []
                        strlstFields = arcpy.ListFields(str_merge)
                        for field in strlstFields:
                            strfieldnames.append(field.name)
                        if str(str_cur_fields[0]) in strfieldnames:
                            print(f'{str_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print(f'Adding {str_cur_fields[0]} field to streams dataset {str_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(str_merge, str(str_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(str_merge, str_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0', '')
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
                        if str(str_cur_fields[2]) in strfieldnames:
                            print(f'{str_cur_fields[2]} field already in dataset {str_merge}')
                            print('----------')
                        else:
                            print(f'Adding {str_cur_fields[2]} field to streams dataset {str_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(str_merge, str(str_cur_fields[2]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(str_merge, str_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) + '_' + strval.replace(".0", "")
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)

                    # Set merged VAA Table dataset
                    elif 'vaa_merge' == filename:
                        vaasource = os.path.join(dirpath, filename)
                        vaaname = roi + '_' + filename
                        vaa_path = os.path.join(outgdb, vaaname)
                        if not arcpy.Exists(vaa_path):
                            print(f'Copying {vaaname}...')
                            print('----------')
                            # Make local copy projected in AKAlbers
                            vaa_merge = arcpy.TableToTable_conversion(vaasource, outgdb, vaaname)
                        else:
                            print(f'Merged VAA Table dataset {vaa_path} already created...')
                            print('----------')
                            vaa_merge = vaa_path
                        vaas_lst.append(vaa_merge)
                        vaafieldnames = []
                        vaalstFields = arcpy.ListFields(vaa_merge)
                        for field in vaalstFields:
                            vaafieldnames.append(field.name)
                        if str(str_cur_fields[0]) in vaafieldnames:
                            print(f'{str_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print(f'Adding {str_cur_fields[0]} field to VAA Table dataset {vaa_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(vaa_merge, str(str_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(vaa_merge, str_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0', '')
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
                        if str(str_cur_fields[2]) in vaafieldnames:
                            print(f'{str_cur_fields[2]} field already in dataset {vaa_merge}')
                            print('----------')
                        else:
                            print(f'Adding {str_cur_fields[2]} field to VAA Table dataset {vaa_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(vaa_merge, str(str_cur_fields[2]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(vaa_merge, str_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) + '_' + strval.replace(".0", "")
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
                    # Set merged watersheds dataset
                    elif 'wtds_merge' == filename:
                        wtdsource = os.path.join(dirpath, filename)
                        wtdname = roi + '_' + filename
                        wtdpath = os.path.join(outgdb, wtdname)

                        if not arcpy.Exists(wtdpath):
                            print(f'Copying {wtdname}...')
                            print('----------')
                            # Make local copy projected in AKAlbers
                            wtd_merge = arcpy.FeatureClassToFeatureClass_conversion(wtdsource, outgdb, wtdname)
                        else:
                            print(f'Merged watershed dataset {wtdpath} already created...')
                            print('----------')
                            wtd_merge = wtdpath
                        wtds_lst.append(wtd_merge)
                        wtdfieldnames = []
                        wtdlstFields = arcpy.ListFields(wtd_merge)
                        for field in wtdlstFields:
                            wtdfieldnames.append(field.name)
                        if str(wtd_cur_fields[0]) in wtdfieldnames:
                            print(f'{wtd_cur_fields[0]} field already in dataset')
                            print('----------')
                        else:
                            print(f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_txt field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0', '')
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
                        if str(wtd_cur_fields[2]) in wtdfieldnames:
                            print(f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                            print('----------')
                        else:
                            print(f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                            print('----------')
                            # add cat_ID_con field and concat cat_ID + region
                            arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) + '_' + strval.replace(".0", "")
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' == filename:
                        # Make local copy projected in AKAlbers
                        catssource = os.path.join(dirpath, filename)
                        catsname = roi + "_" + filename
                        catspath = os.path.join(outgdb, catsname)
                        if not arcpy.Exists(catspath):
                            print(f'Copying {catsname}...')
                            print('----------')
                            cats = arcpy.FeatureClassToFeatureClass_conversion(catssource, outgdb, catsname)
                        else:
                            print(f'{catspath} already created')
                            print('----------')
                            cats = catspath
                        cats_lst.append(cats)
                        catlstfields = arcpy.ListFields(cats)
                        catfieldnames = []
                        for field in catlstfields:
                            catfieldnames.append(field.name)
                        if str(cat_cur_fields[0]) in catfieldnames:
                            print(f'{cat_cur_fields[0]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print(f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field
                            arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                            # populate cat_ID_txt
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[0] = strval.replace('.0', '')
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
                        if str(cat_cur_fields[2]) in catfieldnames:
                            print(f'{cat_cur_fields[2]} field already in dataset {cats}')
                            print('----------')
                        else:
                            print(f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                            print('----------')
                            # add cat_ID_txt field & cat_ID + region concat field
                            arcpy.AddField_management(cats, str(cat_cur_fields[2]), field_type='TEXT')
                            # populate cat_ID_con
                            with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                                for row in cur:
                                    strval = str(row[1])
                                    row[2] = str(roi) + '_' + strval.replace(".0", "")
                                    # Update rows
                                    cur.updateRow(row)
                                del (row)
                            del (cur)
print(cats_lst)
print(site_lst)
print(vaas_lst)
print(streams_lst)
print(wtds_lst)

## Merge collected datasets

In [ ]:
import time
import datetime

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

outcats = os.path.join(outgdb, "all_cats_int_merge")
outsites = os.path.join(outgdb, "all_sites_sj_merge")
outvaa = os.path.join(outgdb, "all_vaas_merge")
outwtd = os.path.join(outgdb, "all_wtds_merge")
outstreams = os.path.join(outgdb, "all_taustreams_merge")
print(f"Merging dataset {outsites}...\n '----------' ")
sitesmerge = arcpy.Merge_management(site_lst, outsites, "", "ADD_SOURCE_INFO")
print(f"Merging dataset {outcats}...\n '----------' ")
catsmerge = arcpy.Merge_management(cats_lst, outcats, "", "ADD_SOURCE_INFO")
print(f"Merging dataset {outvaa}...\n '----------' ")
vaasmerge = arcpy.Merge_management(vaas_lst, outvaa, "", "ADD_SOURCE_INFO")
print(f"Merging dataset {outstreams}...\n '----------' ")
streamsmerge = arcpy.Merge_management(streams_lst, outstreams, "", "ADD_SOURCE_INFO")
print(f"Merging dataset {outwtd}...\n '----------' ")
wtdsmerge = arcpy.Merge_management(wtds_lst, outwtd, "", "ADD_SOURCE_INFO")

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

## Buffer and clip sites
Buffer sites by 30 meters and loop through each site and clip to its respective catchment.  Merge all clips together
when complete

In [ ]:
import arcpy
import os
from arcpy.sa import *
import time
import datetime

# Setting outgdb to location created in Covariate notebook but any temp gdb will work
outgdb = r"D:\\GIS_temp\\AKSSF_land_met\\AKSSF_land_met.gdb"
# Set data dir = to folder with regional subfolders and gdbs
data_dir = r"D:\GIS_Temp\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions = arcpy.ListWorkspaces(workspace_type="Folder")
#buffer value of 30 meters
buffval = 30
bufpath = os.path.join(outgdb, "sites_buffer")
buffer = arcpy.Buffer_analysis(sitesmerge, bufpath, "30 meters")
# this makes sure the buffer does not extend outside of the catchment - cannot use memory/ object as input for zonal stats
# must write to gdb
clips = []
# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()
buflyr = arcpy.MakeFeatureLayer_management(bufpath, "buflyr")
catlayer = arcpy.MakeFeatureLayer_management(catsmerge, 'catlayer')
fieldlist = []
fields = arcpy.ListFields(bufpath)
for field in fields:
    fieldlist.append(field.name)
row_count = len(list(i for i in arcpy.da.SearchCursor(bufpath, fieldlist)))
print(row_count)

c = 1
with arcpy.da.SearchCursor(bufpath, ['SiteID', 'cat_ID_con', 'OBJECTID']) as cur:
    for row in cur:
        fieldValue = str(row[0])
        if fieldValue.find("'") != -1:
            newVal = fieldValue.replace("'", "''")
            print(f'Apostrophe found in {fieldValue}, replacing with {newVal}')
        else:
            print(f'Site ID: {fieldValue} OK')
            newVal = fieldValue

        clause = """ "cat_ID_con" = '%s'""" % row[1]
        clause1 = """ "SiteID" = '%s'""" % newVal
        print(f' {c} of {row_count} -  Clipping {row[0]} to {row[1]}...')
        print('----------')
        bufselect = arcpy.SelectLayerByAttribute_management(buflyr, 'NEW_SELECTION', where_clause=clause1)
        catselect = arcpy.SelectLayerByAttribute_management(catlayer, 'NEW_SELECTION', where_clause=clause)
        clipname = 'clip_' + str(c)
        clipout = os.path.join(outgdb, clipname)
        clip = arcpy.Clip_analysis(bufselect, catselect, clipout)
        clips.append(clipout)
        c += 1
    del (row)
del (cur)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

In [ ]:
# Merge Clips and delete individual clipped buffers
clip_path = os.path.join(outgdb, 'all_buffsites_clip')
clip_outfile = arcpy.Merge_management(clips, clip_path)

delete = clips
for d in delete:
    arcpy.DeleteFeatures_management(d)


In [ ]:
#zonal stats on buffer intersection with flow accumulation grids (need to loop through regions for this)
# to get maximum flow accumulation bc some points may not exactly fall on stream grid
tbl_lst = []
# Seperate data by
nhdplus_dat = ['Cook_Inlet', 'Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat

for roi in rois:
    # Loop through regional folders
    # Grab stream order from VAA/Streams and join to sites_sj
    for region in regions:
        arcpy.env.workspace = region
        raslist = arcpy.ListRasters()
        if roi in str(region):
            if roi in nhdplus_dat:
                cat_cur_fields = ['cat_ID_txt', 'NHDPlusID', "cat_ID_con"]
                cf = 25
                # grab stream order from vaa - upstream and downstream lengths - and stream gradient
                vaa = 'vaa_merge'
                print(f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and spatial join fields '
                      f'{sj_cur_fields}')
                print('----------')
            # Set data and variables unique to regions with TauDEM Data
            elif roi in tauDem_dat:
                # grab stream order from tauDEM - upstream and downstream lengths - and stream gradient
                streams = "streams_merge"
                # Fields for update cursor
                if roi == 'Bristol_Bay':
                    # Fields for update cursor
                    cat_cur_fields = ['cat_ID_txt', 'catID', "cat_ID_con"]
                else:
                    cat_cur_fields = ['cat_ID_txt', 'gridcode', "cat_ID_con"]
                cf = 100
                print(f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and spatial join fields '
                      f'{sj_cur_fields}')
                print('----------')
            print(f'{roi} using data from {region} folder')
            print('--------')
            fac = os.path.join(region, 'fac.tif')
            if arcpy.Exists(fac):
                print(f'Flow accumulation grid {fac} selected for {roi}')
            else:
                print(f'No flow accumulation raster for {region}')
            outtable = os.path.join(outgdb, (roi + "_maxfac"))
            print(f'Calculating {roi} catchment Max flow accumulation stats...')
            arcpy.env.snapRaster = fac
            arcpy.env.cellSize = fac
            try:
                field = 'cat_ID_con'
                operator = 'LIKE'
                value = roi
                sba = """"{}" {} '{}%'""".format(field, operator, value)
                clip_select = arcpy.SelectLayerByAttribute_management(clip_outfile, 'NEW_SELECTION', sba)
                zon_start = time.time()
                cat_facMax_table = ZonalStatisticsAsTable(in_zone_data=clip_select,
                                                          zone_field=cat_cur_fields[2],
                                                          in_value_raster=fac,
                                                          out_table=outtable,
                                                          statistics_type='MAXIMUM'
                                                          )
                # Add region identifier field for catchment table
                arcpy.AddField_management(cat_facMax_table, 'region', field_type='TEXT')
                # Add cat_ID_Con field
                arcpy.AddField_management(cat_facMax_table, 'cat_ID_txt', field_type='TEXT')
                # Update fields
                # Add upstream area as calculated from Max flow acc zonal stats for catchment table
                arcpy.AddField_management(cat_facMax_table, 'site_acc_sqKm', field_type='DOUBLE')

                with arcpy.da.UpdateCursor(cat_facMax_table,
                                           ['region', 'cat_ID_txt', 'cat_ID_con', 'MAX', 'site_acc_sqKm']) as cur:
                    for row in cur:
                        strval = str(row[1])
                        row[0] = roi
                        row[1] = strval.replace('.0', '')
                        row[4] = (cf * row[3]) / 1000000
                        # Update
                        cur.updateRow(row)
                    del (row)
                del (cur)
                # rename MAX field
                arcpy.AlterField_management(cat_facMax_table, "MAX", 'site_max_acc', 'site_max_acc')
                zon_stop = time.time()
                zon_time = int(zon_stop - zon_start)
                print(f'Max Flow zonal stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zon_time)})')
                print('----------')

                tbl_lst.append(outtable)
            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

In [ ]:
# Merge flow accumulation outtables
out_table_merge = os.path.join(outgdb, "AKSSF_site_sj_maxfac")
flowmerge = arcpy.Merge_management(tbl_lst, out_table_merge, "", "ADD_SOURCE_INFO")
print("Merge all maxfac tables completed")
# Merge watersheds and recalculate area_km2 to join back to sites sj - Not all sites had these fields joined
wtds_merge_list = []
arcpy.env.workspace = outgdb
for fc in arcpy.ListFeatureClasses('*wtds_merge'):
    print(fc)
    arcpy.CalculateField_management(fc, "Area_km2", expression="!SHAPE.area@SQUAREKILOMETERS!")
    fieldlist = arcpy.ListFields(fc)
    wtds_merge_list.append(fc)

wtds_merge_path = os.path.join(outgdb, "all_akssf_wtds")
wtds_merge = arcpy.Merge_management(wtds_merge_list, wtds_merge_path, "", "ADD_SOURCE_INFO")

## Add stream metrics from VAA table and streams merge
### VAA ATTRIBUTES
* ARBOLATESU takes advantage of the LengthKM field in NHDFlowline, which measures the length of the flowline in kilometers. ARBOLATESU is determined by accumulating the length of all the upstream flowlines from the bottom of the current flowline and so provides the total length of the upstream drainage network from the bottom of the current flowline.
* ARBOLATESU is the total length of the upstream drainage network from the bottom of the current flowline

* The PATHLENGTH is the distance from the bottom of a flowline to the bottom of the terminal flowline along the main path as identified in the TERMINALPA field. There may be many pathways between a flowline and the terminal flowline because of divergences in the network so PATHLENGTH is computed by following the main path at each divergence.
   * PATHLENGTH is the distance from the bottom of a flowline to the end of the network and is calculated on the main network path
   * PATHLENGTH at the terminal Flowline (Isolated, Sinks, Oceans) is equal to zero.

* This is what you need to know about STARTFLAG AND TERMINALFL:
  * STARTFLAG identifies headwater flowlines
  * TERMINALFL identifies network end flowlines
  * Values of 0 indicate the feature is not headwater (STARTFLAG) or network end (TERMINALFL)
  * values of 1 indicate the feature is a headwater (STARTFLAG) or network end (TERMINALFL)

* TotalDrainageAreaSqKm
  * Total cumulative area, in square kilometers
* AreaSqKm - Catchment area, in square kilometers
* Slope - Slope of the flowline from smoothed elevation (unitless)
### tauDEM Attributes
* DSContArea - Drainage area at the downstream end of the link. Generally this is one grid cell upstream of the downstream end because the drainage area at the downstream end grid cell includes the area of the stream being joined.
* USContArea - Drainage area at the upstream end of the link
* Slope - Average slope of the link (computed as drop/length)
* DOUT_END - Distance to the outlet from the downstream end of the link
* DOUT_START - Distance to the outlet from the upstream end of the link


In [ ]:
#Join fields from flow acc zonal stats and wtds merge to compare watershed area
delete_fields = ['StreamOrde', 'strmOrder', 'Slope', 'Slope_1', 'HUC8_ID', 'HUC8', 'Join_Count', 'TARGET_FID',
                 'proc_reg']
flow_join_fields = ['site_max_acc', 'site_acc_sqKm']
wtds_join_fields = ['Area_km2']
vaa_join_fields = ['StreamOrde', 'Slope', 'AreaSqKm', 'ArbolateSu', 'PathLength', 'StartFlag', 'TerminalPa',
                   'TotalDrainageAreaSqKm']
streams_join_fields = ['strmOrder', 'Slope', 'USContArea', 'DSContArea', 'LINKNO', 'DOUTEND', 'DOUTSTART']
#join flow acc
arcpy.JoinField_management(sitesmerge, sj_cur_fields[2], out_table_merge, sj_cur_fields[2], flow_join_fields)
#join wtds merge
arcpy.JoinField_management(sitesmerge, sj_cur_fields[2], wtds_merge, sj_cur_fields[2], wtds_join_fields)
#join NHDPlus vaa table data
arcpy.JoinField_management(sitesmerge, sj_cur_fields[2], vaasmerge, str_cur_fields[2], vaa_join_fields)
# join TauDEM streams merge data
arcpy.JoinField_management(sitesmerge, sj_cur_fields[2], streamsmerge, str_cur_fields[2], streams_join_fields)
# Add fields as necessary
arcpy.AddField_management(sitesmerge, "area_diff", field_type="DOUBLE")
arcpy.AddField_management(sitesmerge, "str_slope", field_type="DOUBLE")
arcpy.AddField_management(sitesmerge, "str_ord", field_type="SHORT")

# Make list of fields to keep
str_fields = []
for field in arcpy.ListFields(sitesmerge):
    str_fields.append(field.name)
keep_fields = [item for item in str_fields if item not in delete_fields]
with arcpy.da.UpdateCursor(sitesmerge,
                           ["area_diff", 'Area_km2', 'site_acc_sqKm', 'str_slope', 'Slope', 'Slope_1', 'str_ord',
                            'StreamOrde', 'strmOrder']) as cur:
    for row in cur:
        # Set conditions for na values in vaa/tau slope
        if row[4] != None:
            slope = row[4]
        else:
            slope = row[5]
        # Set conditions for na values in vaa/tau strOrder
        if row[7] != None:
            ord = row[7]
        else:
            ord = row[8]
        row[3] = slope
        row[6] = ord
        # Calculate difference in area as watershed polygon area in sqKm minus area calculated from max flow accumulation in sqKm
        row[0] = row[1] - row[2]
        cur.updateRow(row)
    del (row)
del (cur)
print('Join Fields Complete')
print('----------')
# Calculate lat/lon in Nad83
# arcpy.management.CalculateGeometryAttributes("all_sites_sj_merge", "lat POINT_Y;lon POINT_X", '', '','GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "DD")

In [ ]:
# Delete unecessary fields
for field in arcpy.ListFields(sitesmerge):
    if field.name not in keep_fields:
        arcpy.DeleteField_management(sitesmerge, field.name)
        print(f'Dropping field: {field.name}')

In [ ]:
#Export CSV to read into R
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
sj_outname = 'AKSSF_sites_sj_maxfac.csv'
arcpy.conversion.TableToTable(sitesmerge, outdir, sj_outname)
print('CSV export complete')
print('----------')

In [ ]:
# Must remove fields with null values or give null meaning values for numpy array
# Make site merge df
sitemerge_df = pd.DataFrame()
fields = ['SiteID', 'cat_ID_con', 'area_diff', 'lat', 'lon', 'site_max_acc', 'site_acc_sqKm', 'Area_km2', 'str_slope',
          'str_ord']
#fields = keep_fields
sitesmerge_arr = arcpy.da.TableToNumPyArray(sitesmerge, fields)
sitemerge_df = pd.DataFrame(sitesmerge_arr)
sitemerge_df = sitemerge_df.set_index('cat_ID_con')

sitemerge_df